<a href="https://colab.research.google.com/github/edurso/tfod-wkspc/blob/master/notebooks/tensorflow_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
WORKDIR='/tf/data/MyDrive/tensorflow/power-port-targeting/' # end /
MODEL='ssd_resnet50_v1_fpn_640x640_coco17_tpu-8' # needs to be same in pre-trained-models and models
IMG=WORKDIR+'images/test/frame-321.jpg'# test image

In [ ]:
!rm -rf /tf/models /tf/workspace
!mkdir -p /tf/data
from google.colab import drive
import os
drive.mount('/tf/data')

In [ ]:
with open('/model', 'w') as m:
    m.write(MODEL)
with open('/workdir', 'w') as w:
    w.write(WORKDIR)

Setup Workspace

In [ ]:
%%bash

# install deps 
python --version
pip install -U --pre tensorflow=="2.*"
pip install tf_slim pycocotools

In [ ]:
%%bash

# clear previous runs
mkdir -p /tf/models/ /tf/workspace/

# clone repos
git clone --depth 1 https://github.com/tensorflow/models /tf/models
git clone --depth 1 https://github.com/edurso/tfod-wkspc /tf/workspace

In [ ]:
%%bash

# compile protobufs
cd /tf/models/research/
protoc object_detection/protos/*.proto --python_out=.

# install object detector package
cp /tf/models/research/object_detection/packages/tf2/setup.py . 
pip install .

In [ ]:
%%bash

# test installation
cd /tf/models/research/object_detection/builders/
python model_builder_tf2_test.py

In [ ]:
import tensorflow as tf
print(tf.__version__)

Now, go to the [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) and copy a model to `pre-trained-models/`. 

Split the dataset and copy images to `images/test/` and `images/train/`.

Add `label_map.pbtxt` to `annotations/`.

Make a new directory in `models/` named similarly (or the same) to the model you downloaded from the zoo. Copy the `pipeline.config` file from the model you downloaded to this directory.

Verify the colab runtime is configured for a GPU.

Now update the `pipeline.config` in `models/` to refelect your dataset.

In [ ]:
os.chdir(WORKDIR)

# Generate TFRecord for Training Data
!python /tf/workspace/scripts/preprocessing/generate-tfrecord.py \
    -x images/train \
    -l annotations/label_map.pbtxt \
    -o annotations/train.record

# Generate TFRecord for Validation Data
!python /tf/workspace/scripts/preprocessing/generate-tfrecord.py \
    -x images/test \
    -l annotations/label_map.pbtxt \
    -o annotations/test.record

In [ ]:
os.chdir(WORKDIR)
%load_ext tensorboard
%tensorboard --logdir=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

In [ ]:
os.chdir(WORKDIR)
!python /tf/models/research/object_detection/model_main_tf2.py \
    --include masks \
    --model_dir=models/$(cat /model) \
    --pipeline_config_path=models/$(cat /model)/pipeline.config

In [ ]:
os.chdir(WORKDIR)
!python /tf/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path ./models/$(cat /model)/pipeline.config \
    --trained_checkpoint_dir ./models/$(cat /model)/ \
    --output_directory ./exported-models/trained_model

In [ ]:
model_dir = WORKDIR+'exported-models/trained_model/saved_model'
output_names = ['StatefulPartitionedCall']
save_pb_model_path = model_dir+'/freeze_graph.pb'

def convert_saved_model_to_pb(output_node_names, input_saved_model_dir, output_graph_dir):
    from tensorflow.python.tools import freeze_graph

    output_node_names = ','.join(output_node_names)

    freeze_graph.freeze_graph(input_graph=None, input_saver=None,
                              input_binary=None,
                              input_checkpoint=None,
                              output_node_names=output_node_names,
                              restore_op_name=None,
                              filename_tensor_name=None,
                              output_graph=output_graph_dir,
                              clear_devices=None,
                              initializer_nodes=None,
                              input_saved_model_dir=input_saved_model_dir)

convert_saved_model_to_pb(output_names, model_dir, save_pb_model_path)

In [ ]:
import cv2 as cv

model=WORKDIR+'exported-models/trained_model/saved_model/freeze_graph.pb'
labels=WORKDIR+'annotations/label_map.pbtxt'

cvNet = cv.dnn.readNetFromTensorflow(model, labels)

img = cv.imread(IMG)
rows = img.shape[0]
cols = img.shape[1]
cvNet.setInput(cv.dnn.blobFromImage(img, size=(640, 640), swapRB=True, crop=False))
cvOut = cvNet.forward()

for detection in cvOut[0,0,:,:]:
    score = float(detection[2])
    if score > 0.3:
        left = detection[3] * cols
        top = detection[4] * rows
        right = detection[5] * cols
        bottom = detection[6] * rows
        cv.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), (23, 230, 210), thickness=2)

cv.imshow('img', img)
cv.waitKey()